In [ ]:
import k3d
import numpy as np
import sympy as sp
sp.init_printing()
from bmcs_utils.api import Cymbol

In [ ]:
f_t = Cymbol(r'f_\mathrm{Nt}', codename='f_t_')
f_c = Cymbol(r'f_\mathrm{Nc}', codename='f_c_')
f_c0 = Cymbol(r'f_\mathrm{Nc0}', codename='f_c0_')
f_s = Cymbol(r'f_\mathrm{T}', codename='f_s_')
m = Cymbol(r'm', codename='m_', real=True, nonnegative=True)
mparams = (f_s, m, f_t, f_c, f_c0)
mparams

In [ ]:
import sympy as sp

# Define the components of the stress tensor
sigma_xx, sigma_yy, sigma_xy, sigma_xz, sigma_yz, sigma_zz = \
    sp.symbols('sigma_xx sigma_yy sigma_xy sigma_xz sigma_yz sigma_zz')

# Create the stress tensor as a 2x2 matrix
sig = sp.Matrix([[sigma_xx, sigma_xy, sigma_xz],
                           [sigma_xy, sigma_yy, sigma_yz],
                           [sigma_xz, sigma_yz, sigma_zz]])


In [ ]:
# Define the components of the stress tensor
sigma_ax, sigma_rad = \
    sp.symbols('sigma_ax sigma_rad')
# Create the stress tensor as a 2x2 matrix
sig_axsym = sp.Matrix([[sigma_ax, 0, 0],
                [0, sigma_rad, 0],
                [0, 0, sigma_rad]])

In [ ]:
# Calculate the first invariant (I1)
I1_axsym_ = sig_axsym.trace()
sig_axsym_vol = I1_axsym_ / 3 * sp.eye(3)

# Calculate the second invariant (I2)
sig_axsym_dev = sig_axsym - sig_axsym_vol 

J2_axsym_ = -sp.simplify(sp.Rational(1,2) * 
                         (sig_axsym_dev.trace()**2 - (sig_axsym_dev.T * sig_axsym_dev).trace()))

In [ ]:
J2_axsym_

In [ ]:

sigma_1, sigma_2, sigma_3 = \
    sp.symbols('sigma_1 sigma_2 sigma_3')

# Create the stress tensor as a 2x2 matrix
sig = sp.Matrix([[sigma_1, 0, 0],
                           [0, sigma_2, 0],
                           [0, 0, sigma_3]])

# Calculate the first invariant (I1)
I1_ = sig.trace()
I2_ = sp.simplify(sp.Rational(1,2) * (sig.trace()**2 - (sig.T * sig).trace()))
I3_ = sp.simplify(sp.det(sig))
sig_vol = I1_ / 3 * sp.eye(3)

# Calculate the second invariant (I2)
sig_dev = sig - sig_vol 

J2_ = -sp.simplify(sp.Rational(1,2) * (sig_dev.trace()**2 - (sig_dev.T * sig_dev).trace()))
J3_ = sp.det(sig_dev)

In [ ]:
I1_, I2_, I3_

In [ ]:
J2_, J3_

In [ ]:
sp.factor(J2_)

In [ ]:
get_J2 = sp.lambdify((sigma_1, sigma_2, sigma_3), J2_)
sp.sqrt(get_J2(-97, 0, 0))# , get_sqrt_J2_paper(-97, 0, 0)

In [ ]:
I1, J2, J3 = sp.symbols(r'I1, J2, J3')
a, lam, b = sp.symbols(r'a, lambda, b')
f_co = sp.symbols(r'f_co')

In [ ]:
%%capture
%run threshold_function_f_df.ipynb

In [ ]:
f_ = (f_solved_
      .subs({x: I1, y: sp.sqrt(J2)})
     )
f_

In [ ]:
f_I1_J2_J3_ = f_.subs({I1: I1_, J2: J2_})

In [ ]:
get_f = sp.lambdify((sigma_1, sigma_2, sigma_3) + mparams, f_I1_J2_J3_ )

In [ ]:
_mparams = dict(
    f_t_=6,
    f_s_= 3,
    f_c0_=20,
    f_c_=60,
    m_ = 0.2
)

In [ ]:
# sourcery skip: avoid-builtin-shadow
sig_min, sig_max, n_sig  = -30, 5, 80
range = np.linspace(sig_min, sig_max, n_sig)
sig_1, sig_2, sig_3 = np.meshgrid(range, range, range)
get_f(-10, -10, -10, **_mparams )

In [ ]:
f_range = get_f(sig_1, sig_2, sig_3, **_mparams )

In [ ]:
f_range[np.isnan(f_range)] = -1

In [ ]:

plt_marching = k3d.marching_cubes(np.array(f_range, dtype=np.float32), level=0.0,
                                  color=0x0e2763,
                                  opacity=0.25,
                                  xmin=sig_min*1.2, xmax=sig_max*1,
                                  ymin=sig_min*1.2, ymax=sig_max*1,
                                  zmin=sig_min*1.2, zmax=sig_max*1,
                                  compression_level=9,
                                  flat_shading=False)

plot = k3d.plot()
plot += plt_marching
plot.display()

In [ ]:
'''
Created on 22.06.2016

@author: Yingxiong

the Willam-Warnke yield surface, 
https://en.wikipedia.org/wiki/Willam-Warnke_yield_criterion
'''
from __future__ import division

import matplotlib.pylab as plt
import numpy as np


def rho_3(xi, theta):
    '''three parameter model'''

    sig_c = 1.
    sig_t = 0.3
    sig_b = 1.7

    rc = np.sqrt(6 / 5) * sig_b * sig_t / \
        (3 * sig_b * sig_t + sig_c * (sig_b - sig_t))
    rt = np.sqrt(6 / 5) * sig_b * sig_t / (sig_c * (2 * sig_b + sig_t))

    u = 2 * rc * (rc ** 2 - rt ** 2) * np.cos(theta)

    a = 4 * (rc ** 2 - rt ** 2) * np.cos(theta) ** 2 + \
        5 * rt ** 2 - 4 * rt * rc
    v = rc * (2 * rt - rc) * np.sqrt(a)

    w = 4 * (rc ** 2 - rt ** 2) * np.cos(theta) ** 2 + (rc - 2 * rt) ** 2

    r = (u + v) / w
    z = sig_b * sig_t / sig_c / (sig_b - sig_t)

    lambda_bar = 1 / np.sqrt(5) / r
    B_bar = 1 / np.sqrt(3) / z

    return -(B_bar * xi - sig_c) / lambda_bar


def rho_5(xi, theta):
    '''five parameter model'''

    ft = 0.5 * np.sqrt(3)  # uniaxial tensile strength
    fcu = 6. * np.sqrt(3)  # uniaxial compressive strength
    fcb = 10. * np.sqrt(3)  # biaxial compressive strength

    a_z = ft / fcu
    a_u = fcb / fcu

    x = 3.67
    q1 = 1.59
    q2 = 1.94

    a2_numerator = np.sqrt(6 / 5) * x * (a_z - a_u) - \
        np.sqrt(6 / 5) * a_z * a_u + q1 * (2 * a_u + a_z)
    a2_denominator = (2 * a_u + a_z) * (x ** 2 - 2 / 3. *
                                        a_u * x + 1 / 3. * a_z * x - 2 / 9. * a_z * a_u)
    a2 = a2_numerator / a2_denominator
    a1 = 1 / 3. * (2 * a_u - a_z) * a2 + np.sqrt(6 / 5) * \
        (a_z - a_u) / (2 * a_u + a_z)
    a0 = 2 / 3. * a_u * a1 - 4 / 9. * a_u ** 2 * a2 + np.sqrt(2 / 15.) * a_u

    x0 = (-a1 - np.sqrt(a1 ** 2 - 4 * a0 * a2)) / (2 * a2)

    b2 = (q2 * (x0 + 1 / 3) - np.sqrt(2 / 15.) * (x0 + x)) / \
        ((x + x0) * (x - 1 / 3.) * (x0 + 1 / 3.))
    b1 = (x + 1 / 3) * b2 + (np.sqrt(6 / 5) - 3 * q2) / (3 * x - 1)
    b0 = -x0 * b1 - x0 ** 2 * b2

    r1 = a0 + a1 * (xi / fcu) + a2 * (xi / fcu) ** 2
    r2 = b0 + b1 * (xi / fcu) + b2 * (xi / fcu) ** 2

    r_numerator = 2 * r2 * (r2 ** 2 - r1 ** 2) * np.cos(theta) + r2 * (2 * r1 - r2) * \
        np.sqrt(4 * (r2 ** 2 - r1 ** 2) * np.cos(theta)
                ** 2 + 5 * r1 ** 2 - 4 * r1 * r2)
    r_denominator = 4 * (r2 ** 2 - r1 ** 2) * \
        np.cos(theta) ** 2 + (r2 - 2 * r1) ** 2
    r = r_numerator / r_denominator
    return r * fcu





In [ ]:

x_lower_limit = -30.
x_upper_limit = 8.

xi, theta = np.mgrid[x_lower_limit:x_upper_limit:100j,
                        0:np.pi / 3:20j]

# the symmetry of the yielding surface (0<theta<pi/3)
theta = np.hstack(
    (theta, theta[:, ::-1],
        theta, theta[:, ::-1],
        theta, theta[:, ::-1]))
xi = np.hstack((xi, xi, xi, xi, xi, xi))
r = rho_5(xi, theta)
r[r < 0] = 0

# the actual coordinates in Haigh-Westergaard coordinates
xi, theta = np.mgrid[x_lower_limit:x_upper_limit:100j,
                        0:2 * np.pi:120j]

sig1 = 1 / np.sqrt(3) * xi + np.sqrt(2. / 3.) * r * np.cos(theta)
sig2 = 1 / np.sqrt(3) * xi + np.sqrt(2. / 3.) * \
    r * -np.sin(np.pi / 6 - theta)
sig3 = 1 / np.sqrt(3) * xi + np.sqrt(2. / 3.) * \
    r * -np.sin(np.pi / 6 + theta)


In [ ]:
p_ = np.rollaxis(np.array([sig1, sig2, sig3]), 0, 3).reshape(-1,3)

In [ ]:
import k3d
plot = k3d.plot()
k3d_points = k3d.points(p_)
plot += k3d_points
plot

In [ ]:
xi.shape

In [ ]:

from numpy import sin

t = np.linspace(-5, 5, 100, dtype=np.float32)
x, y, z = np.meshgrid(t, t, t, indexing='ij')

scalars = sin(x*y + x*z + y*z) + sin(x*y) + sin(y*z) + sin(x*z) - 1

plt_marching = k3d.marching_cubes(scalars, level=0.0,
                                  color=0x0e2763,
                                  opacity=0.25,
                                  xmin=0, xmax=1,
                                  ymin=0, ymax=1,
                                  zmin=0, zmax=1,
                                  compression_level=9,
                                  flat_shading=False)

plot = k3d.plot()
plot += plt_marching
plot.display()

In [ ]:
import mayavi.mlab as mlab
mlab.figure(1, fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))
s = mlab.mesh(sig1, sig2, sig3)  # , scalars=xi)

mlab.axes(s)
mlab.show()
